In [ ]:
import sys
sys.path.extend(['../libs','../src/'])
import dhis_meta as mt
import utils as fn


conf=fn.get_config('/home/n2/.dih.configs/jna.json')
# meta=mt.DHIS_Meta(conf);
# meta.update()

In [ ]:

#!/usr/bin/env python
# apt-get install libbz2-dev ssh-client postgresql-client
# pip install --upgrade pandas openpyxl SQLAlchemy psycopg2-binary google-api-python-client google-auth-httplib2 google-auth-oauthlib oauth2client
# cron_time 0 03 4 * *
import sys
sys.path+=["../libs",'../src']
from main import main 
import utils as fn


if __name__ == "__main__":
    x = sys.argv[1] if len(sys.argv) > 1 else fn.get_month(-1)
    x=fn.get_month(-2)
    data = main(month=x,task_name='jna-dhis',config_file='/home/n2/.dih.configs/jna.json')
    print(data)


In [ ]:
import pandas as pd,json,re,requests as rq,os

base_url='https://script:LhCvuPsm7YDfBQ3@5$12&651@data.d-tree.org'
def prep_key(value):
    if isinstance(value, list):
        return {prep_key(x) for x in value}
    elif isinstance(value, pd.Series):
        return {prep_key(x) for x in value.values}
    elif isinstance(value, str):
        return re.sub(r'\W+', '', value)
    else:
        return value

def get_org_units():
    def set_location(row):
        return prep_key([x['name'] for x in row.ancestors]) | {prep_key(row.orgName)}

    if os.path.exists('.data/organisationUnits.json'):
        with open('.data/organisationUnits.json','r') as file:
            orgs=pd.json_normalize(json.load(file))
    else:
        levels='[5,7,8]'
        root=rq.get(f"{base_url}/api/organisationUnits?fields=id,name,level&filter=name:eq:Zanzibar").json()['organisationUnits'][0]
        url = f"{base_url}/api/organisationUnits?filter=path:like:{root['id']}&filter=level:in:{levels}&fields=id~rename(orgUnit),name~rename(orgName),level,ancestors[name]&paging=false"
        orgs = pd.json_normalize(rq.get(url).json()["organisationUnits"])
        orgs.to_csv('temp_orgs.csv')

    orgs['name_key'] = orgs.orgName.apply(prep_key)
    orgs['location']=orgs.apply(set_location,axis=1)
    return orgs

orgs=get_org_units();
shehia=pd.read_excel('temp_orgs.xlsx','shehia')


In [ ]:

def add_org_unit_id(data: pd.DataFrame):
    def find_matching(x):
        s=orgs[orgs.name_key.isin(x)]
        matches=s[s.location.apply(lambda y:x.issubset(y))]
        return matches.orgUnit.values[0] if matches.size>0 else pd.NA

    loc = [ x for x in data.columns if x in 'shehia,district,supervisory_area']
    data['location']= data[loc].apply(prep_key,axis=1)
    data['orgUnit']=data.location.map(find_matching)
    return data


In [ ]:
import requests as rq

base_url="https://script:LhCvuPsm7YDfBQ3@5$12&651@data.d-tree.org"
data=rq.get(f"{base_url}/api/analytics?dimension=dx:eInfScreMal&filter=pe:202306&tableLayout=true")
data.json()

In [ ]:
import pandas as pd
org_unit_id="gbglobalZZZ"
period="202306"
data_element="eInfScreMal"
data_set="lyqun8ofsVV"
# data=rq.get(f"{base_url}/api/dataValueSets?dataSet={data_set}&orgUnit={org_unit_id}&period={period}&dataElement={data_element}" )
# data.json()


data=rq.get(f"{base_url}/api/organisationUnits?level=8&fields=id&paging=false&filter=id:like:zsh")
orgs=pd.DataFrame(data.json()['organisationUnits'])
# data.json()
data=[]
for ou in orgs.id:
    org=rq.get(f"{base_url}/api/dataValueSets?dataSet={data_set}&orgUnit={ou}&period={period}&dataElement={data_element}" )
    data.append(org)
    # print(org.json())
data

In [ ]:
import sys
sys.path.extend(['../libs','../src/'])
import utils as fn

data=[]
def downloadValues(ids):
    global data
    period="202306"
    data_element="eInfScreMal"
    data_set="lyqun8ofsVV"
    for ou in ids:
        print(ou)
        org=rq.get(f"{base_url}/api/dataValueSets?dataSet={data_set}&orgUnit={ou}&period={period}&filter=dataElement:eq:{data_element}" )
        data.append(org.json())

fn.do_chunks(
    orgs.id.to_list(),
    20,
    func=downloadValues,
    thread_count=8)

data

In [48]:
x=pd.DataFrame()
for n in data:
    x=pd.concat([x,pd.DataFrame(n['dataValues'])])

x['value']=x.value.astype(int)
# x[x.dataElement=='eInfScreMal']
x


,dataElement,period,orgUnit,categoryOptionCombo,attributeOptionCombo,value,storedBy,created,lastUpdated,comment,followup
0,ehddRegistr,202306,zshWEKipang,HllvX50cXC0,HllvX50cXC0,1,script,2023-07-10T10:39:04.000+0000,2023-07-10T10:39:04.000+0000,None,False
1,epplRegistr,202306,zshWEKipang,HllvX50cXC0,HllvX50cXC0,15,script,2023-07-10T10:39:04.000+0000,2023-07-10T10:39:04.000+0000,None,False
2,einfEnrollm,202306,zshWEKipang,geigilUUtWC,HllvX50cXC0,4,script,2023-07-10T10:39:04.000+0000,2023-07-10T10:39:04.000+0000,None,False
3,einfEnrollm,202306,zshWEKipang,LoEgzvgQOmH,HllvX50cXC0,7,script,2023-07-10T10:39:04.000+0000,2023-07-10T10:39:04.000+0000,None,False
4,einfEnrollm,202306,zshWEKipang,YvpoEVXwxtN,HllvX50cXC0,2,script,2023-07-10T10:39:04.000+0000,2023-07-10T10:39:04.000+0000,None,False
...,...,...,...,...,...,...,...,...,...,...,...
297,eInfScrDevD,202306,zshCEUziZZZ,VjqpDvkZ71G,HllvX50cXC0,2,script,2023-10-25T10:10:10.000+0000,2023-10-25T10:10:10.000+0000,None,False
298,eInfScrDevD,202306,zshCEUziZZZ,guUGEmFHyKr,HllvX50cXC0,2,script,2023-10-25T10:10:10.000+0000,2023-10-25T10:10:10.000+0000,None,False
299,eInfScrDevD,202306,zshCEUziZZZ,bHBwhQBCJwR,HllvX50cXC0,8,script,2023-10-25T10:10:10.000+0000,2023-10-25T10:10:10.000+0000,None,False
300,eInfScrDevD,202306,zshCEUziZZZ,YvpoEVXwxtN,HllvX50cXC0,9,script,2023-10-25T10:10:10.000+0000,2023-10-25T10:10:10.000+0000,None,False
